# Desafio Quarentena Dados

O desafio proposto ao final da semana de curso de Data Science promovida pela **Alura** consiste em propor um modelo de Machine Learning para prever as notas de linguagem do ENEM utilizando apenas as outras notas obtidas pelos participantes.

## Leitura dos dados

In [0]:
import pandas as pd
import numpy as np

URI_TREINO = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TESTE = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv"
URI_DESAFIOQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv"

dados_treino = pd.read_csv(URI_TREINO)
dados_teste = pd.read_csv(URI_TESTE)
dados_desafioqt = pd.read_csv(URI_DESAFIOQT)

erro_treino = "Erro ao carregar dados de treino"
erro_teste = "Erro ao carregar dados de teste"
erro_desafioqt = "Erro ao carregar dados de submissão"

assert dados_treino.shape == (150000, 5), erro_treino
assert dados_teste.shape == (20000, 5), erro_teste
assert dados_desafioqt.shape == (10000, 5), erro_desafioqt

In [0]:
coluna_label = 'NU_NOTA_LC'
coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

X_treino = dados_treino[coluna_features].to_numpy()
Y_treino = dados_treino[coluna_label].to_numpy()
X_teste = dados_teste[coluna_features].to_numpy()
Y_teste = dados_teste[coluna_label].to_numpy()

## Construção do Modelo

Construi uma classe que realiza o processo de regressão em duas etapas:
* Primeiro os alunos são classificados em grupos utilizando o método de K-means nas notas obtidas no ENEM;
* Em seguida, ajusta algum modelo de Regressão para cada um dos grupos.

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor


class segmentationRegression:

  def __init__(self):
    self.stdX = StandardScaler()
    self.stdY = StandardScaler()

    self.n_cluster = 5
    self.kmeans = KMeans(n_clusters=self.n_cluster)

    self.clusters = None
    self.clusters_star = None

    self.models = {}
    for i in range(self.n_cluster):
      self.models[str(i)] = MLPRegressor()

    self.fitted = None
    self.forecast = None

  def fit(self, X, Y):
    self.stdX.fit(X)
    self.stdY.fit(Y.reshape(-1, 1))
    X_hat = self.stdX.transform(X)
    Y_hat = self.stdY.transform(Y.reshape(-1, 1))

    self.kmeans.fit(X_hat)
    self.clusters = self.kmeans.labels_

    for i in range(self.n_cluster):
      condition = self.clusters == i
      self.models[str(i)].fit(X_hat[condition],np.ravel(Y_hat[condition]))
    
    temp = []
    for x, cluster in zip(X_hat, self.clusters):
      aux = self.models[str(cluster)].predict(x.reshape(1,4))
      temp.append(self.stdY.inverse_transform(aux).item(0))
    self.fitted = np.array(temp)
    del temp, aux

  def predict(self, X_star):
    X_star_hat = self.stdX.transform(X_star)
    self.clusters_star = self.kmeans.predict(X_star_hat)

    temp = []
    for x, cluster in zip(X_star_hat, self.clusters_star):
      aux = self.models[str(cluster)].predict(x.reshape(1,4))
      temp.append(self.stdY.inverse_transform(aux).item(0))
    self.forecast = np.array(temp)
    del temp, aux

## Aplicação

In [7]:
#Avaliação do modelo
from sklearn.metrics import mean_squared_error

SR = segmentationRegression()
SR.fit(X_treino, Y_treino)
SR.predict(X_teste)
SR_predicoes = SR.forecast

avaliacao_SR = mean_squared_error(Y_teste, SR_predicoes)

print(f"Minha avaliação nos dados de teste foi de {avaliacao_SR}")

Minha avaliação nos dados de teste foi de 2051.1617031544456


In [0]:
#atribuir ao MODELO o nome do seu melhor modelo
from google.colab import files

MODELO = SR
X_desafioqt = dados_desafioqt[coluna_features].to_numpy()
SR.predict(X_desafioqt)
predicao_desafioqt = SR.forecast


desafio_df = pd.DataFrame(dados_desafioqt.ID)
desafio_df[coluna_label] = predicao_desafioqt

#NÃO TROCAR O NOME DO ARQUIVO DE SAÍDA (PREDICAO_DESAFIO)
desafio_df.to_csv('PREDICAO_DESAFIOQT.csv', index=False) 
files.download('PREDICAO_DESAFIOQT.csv')